In [0]:
# #Import packages
# !pip install tqdm
# !pip install bert-tensorflow

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm
import tensorflow_hub as hub
import tensorflow as tf
#import bert.tokenization as tokenization
import tensorflow.keras.backend as K
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil
from tensorflow.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

np.set_printoptions(suppress=True)



The following cell is for bert_tokenization package

In [0]:
import collections
import re
import unicodedata
import six
import tensorflow as tf


def validate_case_matches_checkpoint(do_lower_case, init_checkpoint):
    """Checks whether the casing config is consistent with the checkpoint name."""

    # The casing has to be passed in by the user and there is no explicit check
    # as to whether it matches the checkpoint. The casing information probably
    # should have been stored in the bert_config.json file, but it's not, so
    # we have to heuristically detect it to validate.

    if not init_checkpoint:
        return

    m = re.match("^.*?([A-Za-z0-9_-]+)/bert_model.ckpt", init_checkpoint)
    if m is None:
        return

    model_name = m.group(1)

    lower_models = [
        "uncased_L-24_H-1024_A-16", "uncased_L-12_H-768_A-12",
        "multilingual_L-12_H-768_A-12", "chinese_L-12_H-768_A-12"
    ]

    cased_models = [
        "cased_L-12_H-768_A-12", "cased_L-24_H-1024_A-16",
        "multi_cased_L-12_H-768_A-12"
    ]

    is_bad_config = False
    if model_name in lower_models and not do_lower_case:
        is_bad_config = True
        actual_flag = "False"
        case_name = "lowercased"
        opposite_flag = "True"

    if model_name in cased_models and do_lower_case:
        is_bad_config = True
        actual_flag = "True"
        case_name = "cased"
        opposite_flag = "False"

    if is_bad_config:
        raise ValueError(
            "You passed in `--do_lower_case=%s` with `--init_checkpoint=%s`. "
            "However, `%s` seems to be a %s model, so you "
            "should pass in `--do_lower_case=%s` so that the fine-tuning matches "
            "how the model was pre-training. If this error is wrong, please "
            "just comment out this check." % (actual_flag, init_checkpoint,
                                              model_name, case_name, opposite_flag))


def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")


def printable_text(text):
    """Returns text encoded in a way suitable for print or `tf.logging`."""

    # These functions want `str` for both Python2 and Python3, but in one case
    # it's a Unicode string and in the other it's a byte string.
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text
        elif isinstance(text, unicode):
            return text.encode("utf-8")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")


def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    index = 0
    with tf.io.gfile.GFile(vocab_file, "r") as reader:
        while True:
            token = convert_to_unicode(reader.readline())
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab


def convert_by_vocab(vocab, items):
    """Converts a sequence of [tokens|ids] using the vocab."""
    output = []
    for item in items:
        output.append(vocab[item])
    return output


def convert_tokens_to_ids(vocab, tokens):
    return convert_by_vocab(vocab, tokens)


def convert_ids_to_tokens(inv_vocab, ids):
    return convert_by_vocab(inv_vocab, ids)


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens


class FullTokenizer(object):
    """Runs end-to-end tokenziation."""

    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab = load_vocab(vocab_file)
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)

        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        return convert_by_vocab(self.vocab, tokens)

    def convert_ids_to_tokens(self, ids):
        return convert_by_vocab(self.inv_vocab, ids)


class BasicTokenizer(object):
    """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

    def __init__(self, do_lower_case=True):
        """Constructs a BasicTokenizer.
        Args:
          do_lower_case: Whether to lower case the input.
        """
        self.do_lower_case = do_lower_case

    def tokenize(self, text):
        """Tokenizes a piece of text."""
        text = convert_to_unicode(text)
        text = self._clean_text(text)

        # This was added on November 1st, 2018 for the multilingual and Chinese
        # models. This is also applied to the English models now, but it doesn't
        # matter since the English models were not trained on any Chinese data
        # and generally don't have any Chinese data in them (there are Chinese
        # characters in the vocabulary because Wikipedia does have some Chinese
        # words in the English Wikipedia.).
        text = self._tokenize_chinese_chars(text)

        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if self.do_lower_case:
                token = token.lower()
                token = self._run_strip_accents(token)
            split_tokens.extend(self._run_split_on_punc(token))

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        """Strips accents from a piece of text."""
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text):
        """Splits punctuation on a piece of text."""
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]

    def _tokenize_chinese_chars(self, text):
        """Adds whitespace around any CJK character."""
        output = []
        for char in text:
            cp = ord(char)
            if self._is_chinese_char(cp):
                output.append(" ")
                output.append(char)
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

    def _is_chinese_char(self, cp):
        """Checks whether CP is the codepoint of a CJK character."""
        # This defines a "chinese character" as anything in the CJK Unicode block:
        #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
        #
        # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
        # despite its name. The modern Korean Hangul alphabet is a different block,
        # as is Japanese Hiragana and Katakana. Those alphabets are used to write
        # space-separated words, so they are not treated specially and handled
        # like the all of the other languages.
        if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
                (cp >= 0x3400 and cp <= 0x4DBF) or  #
                (cp >= 0x20000 and cp <= 0x2A6DF) or  #
                (cp >= 0x2A700 and cp <= 0x2B73F) or  #
                (cp >= 0x2B740 and cp <= 0x2B81F) or  #
                (cp >= 0x2B820 and cp <= 0x2CEAF) or
                (cp >= 0xF900 and cp <= 0xFAFF) or  #
                (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
            return True

        return False

    def _clean_text(self, text):
        """Performs invalid character removal and whitespace cleanup on text."""
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)


class WordpieceTokenizer(object):
    """Runs WordPiece tokenziation."""

    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=200):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def tokenize(self, text):
        """Tokenizes a piece of text into its word pieces.
        This uses a greedy longest-match-first algorithm to perform tokenization
        using the given vocabulary.
        For example:
          input = "unaffable"
          output = ["un", "##aff", "##able"]
        Args:
          text: A single token or whitespace separated tokens. This should have
            already been passed through `BasicTokenizer.
        Returns:
          A list of wordpiece tokens.
        """

        text = convert_to_unicode(text)

        output_tokens = []
        for token in whitespace_tokenize(text):
            chars = list(token)
            if len(chars) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            is_bad = False
            start = 0
            sub_tokens = []
            while start < len(chars):
                end = len(chars)
                cur_substr = None
                while start < end:
                    substr = "".join(chars[start:end])
                    if start > 0:
                        substr = "##" + substr
                    if substr in self.vocab:
                        cur_substr = substr
                        break
                    end -= 1
                if cur_substr is None:
                    is_bad = True
                    break
                sub_tokens.append(cur_substr)
                start = end

            if is_bad:
                output_tokens.append(self.unk_token)
            else:
                output_tokens.extend(sub_tokens)
        return output_tokens


def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat in ("Cc", "Cf"):
        return True
    return False


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

# Bert-based Model

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm
import tensorflow_hub as hub
import tensorflow as tf
#import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil
from tensorflow.keras.models import load_model

np.set_printoptions(suppress=True)

In [59]:
# Import data from https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
# there will be a prompt for authentication. Follow the link, click yes, and copy the token

# get name of the files in folder
!ls "/content/gdrive/My Drive/GoogleQA/data/"


Mounted at /content/gdrive
bert  sample_submission.csv  test.csv  train.csv


In [60]:
# Import data
PATH = '/content/gdrive/My Drive/GoogleQA/data/'
BERT_PATH = PATH +'bert/'
tokenizer = FullTokenizer(BERT_PATH+'vocab.txt', True)
MAX_SEQUENCE_LENGTH = 512

df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)
print('sub shape =', df_sub.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5]])
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)
sub shape = (476, 31)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']

In [0]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q, a = _trim_input(t, q, a, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [0]:
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(
            spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.mean(rhos)


class CustomCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_data, test_data, batch_size=16, fold=None):

        self.valid_inputs = valid_data[0]
        self.valid_outputs = valid_data[1]
        self.test_inputs = test_data
        
        self.batch_size = batch_size
        self.fold = fold
        
    def on_train_begin(self, logs={}):
        self.valid_predictions = []
        self.test_predictions = []
        
    def on_epoch_end(self, epoch, logs={}):
        self.valid_predictions.append(
            self.model.predict(self.valid_inputs, batch_size=self.batch_size))
        
        rho_val = compute_spearmanr(
            self.valid_outputs, np.average(self.valid_predictions, axis=0))
        
        print("\nvalidation rho: %.4f" % rho_val)
        
        if self.fold is not None:
            self.model.save_weights(f'bert-base-{fold}-{epoch}.h5py')
        
        self.test_predictions.append(
            self.model.predict(self.test_inputs, batch_size=self.batch_size)
        )

def bert_model():
    
    input_word_ids = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    input_segments = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_segments')
    
    bert_layer = hub.KerasLayer(BERT_PATH, trainable=True)
    
    _, sequence_output = bert_layer([input_word_ids, input_masks, input_segments])
    
    x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(30, activation="sigmoid", name="dense_output")(x)

    model = tf.keras.models.Model(
        inputs=[input_word_ids, input_masks, input_segments], outputs=out)
    
    return model    
        
def train_and_predict(model, train_data, valid_data, test_data, 
                      learning_rate, epochs, batch_size, loss_function, fold):
        
    custom_callback = CustomCallback(
        valid_data=(valid_data[0], valid_data[1]), 
        test_data=test_data,
        batch_size=batch_size,
        fold=None)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=loss_function, optimizer=optimizer)
    model.fit(train_data[0], train_data[1], epochs=epochs, 
              batch_size=batch_size, callbacks=[custom_callback])
    
    return custom_callback

In [63]:
gkf = GroupKFold(n_splits=10).split(X=df_train.question_body, groups=df_train.question_body) ############## originaln_splits=5

#outputs = compute_output_arrays(df_train, output_categories)
#inputs = compute_input_arays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

476it [00:02, 171.23it/s]


## Check the performance of the model on each column

In [64]:
# Split dataset for validation from the train set
df_valid = df_train.sample(n=476)
df_valid_y = df_valid[df_valid.columns[11:]]
df_valid_x = df_valid[df_valid.columns[:11]]

# Convert dataset to input
valid_inputs = compute_input_arays(df_valid_x, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

476it [00:02, 175.70it/s]


In [65]:
df_valid_x.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
5016,7989,Does tagging content affect SEO?,Tagging content (i.e. categorizing content usi...,Ken Liu,https://webmasters.stackexchange.com/users/1837,The tags won't affect the pages directly by be...,John Conde,https://webmasters.stackexchange.com/users/1253,http://webmasters.stackexchange.com/questions/...,TECHNOLOGY,webmasters.stackexchange.com
3762,5989,What is the best introductory Bayesian statist...,Which is the best introductory textbook for Ba...,Shane,https://stats.stackexchange.com/users/5,If I had to choose a single text for a beginne...,Jim Stone,https://stats.stackexchange.com/users/26779,http://stats.stackexchange.com/questions/125/w...,SCIENCE,stats.stackexchange.com
1691,2675,Is this a bug? Angular removes object params w...,Please see this:\n\nhttp://plnkr.co/edit/soubK...,user1804318,https://stackoverflow.com/users/1804318,"Its not bug, its desired behaviourof angular, ...",Milos Mosovsky,https://stackoverflow.com/users/4530419,http://stackoverflow.com/questions/31062344/is...,STACKOVERFLOW,stackoverflow.com
3482,5560,How many creatures should be in a draft deck?,"How many creatures should the average, typical...",JSBձոգչ,https://boardgames.stackexchange.com/users/156,Creatures are what makes you win and what make...,Hackworth,https://boardgames.stackexchange.com/users/1718,http://boardgames.stackexchange.com/questions/...,CULTURE,boardgames.stackexchange.com
51,78,"Is potential energy and ""work done"" the same t...","Is potential energy and ""work done"" the same t...",Uzair,https://physics.stackexchange.com/users/36559,You will have to spent some energy to do work....,Rajath Krishna R,https://physics.stackexchange.com/users/23742,http://physics.stackexchange.com/questions/940...,SCIENCE,physics.stackexchange.com


In [66]:
df_valid_y.head()

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
5016,0.888889,0.777778,0.0,0.666667,1.000000,1.000000,0.666667,0.555556,0.666667,0.0,0.333333,0.666667,0.0,0.0,0.0,0.000000,0.333333,0.666667,1.000000,0.0,0.888889,1.000000,0.666667,1.0,0.888889,0.933333,0.0,0.666667,1.0,0.777778
3762,1.000000,1.000000,1.0,1.000000,0.000000,0.333333,0.666667,0.555556,0.000000,0.0,1.000000,0.333333,0.0,0.0,0.0,0.666667,0.000000,0.000000,0.666667,0.0,1.000000,1.000000,1.000000,1.0,1.000000,0.933333,0.0,0.000000,1.0,1.000000
1691,1.000000,0.666667,0.0,1.000000,1.000000,1.000000,0.555556,0.555556,0.000000,0.0,0.000000,0.666667,0.0,0.0,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.777778,1.000000,0.666667,1.0,1.000000,0.933333,0.0,0.333333,1.0,0.555556
3482,0.666667,0.333333,0.0,0.000000,1.000000,0.000000,0.333333,1.000000,1.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.666667,1.000000,0.666667,1.0,1.000000,0.800000,1.0,0.000000,0.0,1.000000
51,1.000000,0.888889,0.0,0.666667,0.666667,1.000000,0.444444,0.444444,0.333333,0.0,0.333333,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.888889,0.888889,0.666667,1.0,1.000000,0.933333,0.0,0.000000,1.0,0.888889


In [0]:
# Load trained models
#model_path = [f'bert/Full-0.h5', f'bert/bertuned_f0.h5', f'bert/bertuned_f1.h5', f'bert/bertuned_f2.h5', f'bert/bertuned_f3.h5', f'bert/bertuned_f4.h5']
model_path = [BERT_PATH+'bertuned_f0.h5', BERT_PATH+'bertuned_f1.h5', BERT_PATH+'bertuned_f2.h5',BERT_PATH+'bertuned_f3.h5',BERT_PATH+'bertuned_f4.h5']
models = []

for i in range(len(model_path)):
    mp = model_path[i]
    model = bert_model()
    model.load_weights(mp)
    models.append(model)

In [68]:
len(models)

5

In [69]:
# Predict the labels of validation set
valid_predictions = []

for model in models[:1]:
    valid_predictions.append(model.predict(valid_inputs, batch_size=8, verbose=1))

final_predictions_valid = np.mean(valid_predictions, axis=0)

476/476 [==============================] - 24s 51ms/sample


In [70]:
df_valid_pred = pd.DataFrame(final_predictions_valid)
df_valid_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.900936,0.636685,0.008015,0.647269,0.943153,0.956474,0.699755,0.613185,0.919710,0.002984,0.146274,0.844482,0.016048,0.020196,0.031633,0.010348,0.319826,0.314135,0.710218,0.006181,0.831946,0.946026,0.657676,0.973172,0.972096,0.869471,0.149882,0.215696,0.915390,0.896545
1,0.946338,0.821615,0.116703,0.841930,0.606377,0.165957,0.743707,0.537019,0.060573,0.006635,0.568479,0.187824,0.087827,0.012783,0.012591,0.716793,0.015775,0.012439,0.044076,0.003578,0.910398,0.937483,0.810877,0.936941,0.963580,0.886876,0.083372,0.020254,0.202091,0.948778
2,0.781372,0.580100,0.003585,0.859610,0.934892,0.975744,0.600435,0.465962,0.047681,0.006678,0.092950,0.670140,0.009381,0.011022,0.006443,0.006961,0.146667,0.053523,0.570909,0.002577,0.690440,0.944655,0.697969,0.958595,0.982449,0.896239,0.153790,0.095089,0.915511,0.921333
3,0.869121,0.489502,0.017457,0.466302,0.720908,0.552414,0.634979,0.503523,0.924003,0.002369,0.727825,0.101807,0.047216,0.010662,0.015033,0.163110,0.818315,0.097590,0.141463,0.001721,0.801154,0.961856,0.681324,0.976065,0.970115,0.840715,0.863144,0.115209,0.409733,0.939759
4,0.932840,0.804420,0.005942,0.749478,0.972538,0.954982,0.728009,0.633627,0.531264,0.001939,0.060953,0.897610,0.068490,0.015496,0.267029,0.008875,0.003056,0.006596,0.790419,0.004825,0.863293,0.952494,0.596452,0.979766,0.985870,0.842139,0.005526,0.006889,0.966770,0.919844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,0.751196,0.353556,0.004007,0.594660,0.886813,0.945158,0.643245,0.435616,0.064067,0.010862,0.207092,0.016222,0.004638,0.005037,0.009631,0.018084,0.563622,0.111902,0.606537,0.002674,0.661752,0.898347,0.670846,0.957778,0.959994,0.844200,0.499427,0.151637,0.823521,0.878024
472,0.901698,0.548917,0.002574,0.702136,0.905775,0.976769,0.594572,0.542414,0.104331,0.005444,0.210963,0.020617,0.003278,0.006408,0.003644,0.003062,0.828739,0.149893,0.383005,0.001216,0.759161,0.960411,0.723583,0.985729,0.984110,0.899519,0.683371,0.212474,0.916331,0.922766
473,0.828628,0.344384,0.134545,0.246080,0.556089,0.170132,0.686091,0.525992,0.894091,0.009399,0.842937,0.462820,0.695428,0.018750,0.032859,0.032793,0.368557,0.065566,0.405686,0.005681,0.815821,0.920210,0.688353,0.971101,0.946241,0.810855,0.335266,0.050472,0.508325,0.909593
474,0.764244,0.693027,0.007079,0.760026,0.890182,0.952749,0.537249,0.389737,0.242600,0.003701,0.205656,0.960925,0.014490,0.017041,0.004650,0.005269,0.155606,0.039231,0.700660,0.001678,0.795438,0.969791,0.661764,0.978213,0.985048,0.929930,0.584588,0.116188,0.882187,0.935381


In [71]:
df_valid_y

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
5016,0.888889,0.777778,0.000000,0.666667,1.000000,1.000000,0.666667,0.555556,0.666667,0.0,0.333333,0.666667,0.0,0.0,0.0,0.000000,0.333333,0.666667,1.000000,0.0,0.888889,1.000000,0.666667,1.0,0.888889,0.933333,0.000000,0.666667,1.000000,0.777778
3762,1.000000,1.000000,1.000000,1.000000,0.000000,0.333333,0.666667,0.555556,0.000000,0.0,1.000000,0.333333,0.0,0.0,0.0,0.666667,0.000000,0.000000,0.666667,0.0,1.000000,1.000000,1.000000,1.0,1.000000,0.933333,0.000000,0.000000,1.000000,1.000000
1691,1.000000,0.666667,0.000000,1.000000,1.000000,1.000000,0.555556,0.555556,0.000000,0.0,0.000000,0.666667,0.0,0.0,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.777778,1.000000,0.666667,1.0,1.000000,0.933333,0.000000,0.333333,1.000000,0.555556
3482,0.666667,0.333333,0.000000,0.000000,1.000000,0.000000,0.333333,1.000000,1.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.666667,1.000000,0.666667,1.0,1.000000,0.800000,1.000000,0.000000,0.000000,1.000000
51,1.000000,0.888889,0.000000,0.666667,0.666667,1.000000,0.444444,0.444444,0.333333,0.0,0.333333,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.888889,0.888889,0.666667,1.0,1.000000,0.933333,0.000000,0.000000,1.000000,0.888889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4892,0.777778,0.555556,0.000000,1.000000,1.000000,1.000000,0.666667,0.444444,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.333333,1.000000,0.0,0.777778,1.000000,0.666667,1.0,1.000000,0.933333,0.666667,0.333333,1.000000,0.888889
719,0.888889,0.444444,0.000000,0.666667,1.000000,1.000000,0.555556,0.444444,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.666667,0.333333,0.666667,0.0,0.888889,1.000000,0.666667,1.0,1.000000,0.933333,0.666667,0.333333,0.666667,0.888889
5940,0.888889,0.333333,0.666667,0.333333,0.333333,0.000000,0.666667,0.555556,1.000000,0.0,0.666667,0.333333,1.0,0.0,0.0,0.000000,0.666667,0.000000,0.333333,0.0,0.888889,0.888889,0.666667,1.0,0.888889,0.866667,0.666667,0.000000,0.333333,1.000000
1992,0.777778,0.777778,0.000000,0.666667,1.000000,0.666667,0.333333,0.333333,0.333333,0.0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.777778,1.000000,0.666667,1.0,1.000000,1.000000,1.000000,0.000000,0.666667,1.000000


In [72]:
df_test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


### Check the scores using spearmanr from scipy

In [73]:
# performance of each column in the validation dataset, model 1
from scipy.stats import spearmanr

df_spearman = pd.DataFrame(df_valid_y.columns,columns=["Label"])
coefs = []

for i in range(30):
    coef,p = spearmanr(df_valid_pred.iloc[:,i], df_valid_y.iloc[:,i])
    coefs.append(coef)

df_spearman["Score"] = coefs
df_spearman

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


,Label,Score
0,question_asker_intent_understanding,0.555494
1,question_body_critical,0.755898
2,question_conversational,0.471938
3,question_expect_short_answer,0.591397
4,question_fact_seeking,0.644531
5,question_has_commonly_accepted_answer,0.624263
6,question_interestingness_others,0.487013
7,question_interestingness_self,0.605696
8,question_multi_intent,0.763799
9,question_not_really_a_question,0.122518


In [74]:
df_spearman.mean()

Score    0.565329
dtype: float64

The scores of question_not_really_a_question(Label 10, 0.157999), question_type_consequence(Label 14, 0.187766) and question_type_spelling(Label 20, NaN) is significantly lower than others.

After checking the train set, we find the data of these columns is imbalanced. So, we're considering solving this problem using imblearn or other methods.

### Try Multilabel Stratified KFold to tackle umbalanced data

In [75]:
# !pip install six
!pip install iterative-stratification

In [0]:
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
SEED = 42


In [0]:
def bagged_model(valid_inputs):
  valid_predictions = []
  for model in models[:1]:
    valid_predictions.append(model.predict(valid_inputs, batch_size=8, verbose=1))
  final_predictions_valid = np.nanmean(valid_predictions, axis=0)
  return final_predictions_valid

In [0]:
def hstack_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q, a = _trim_input(t, q, a, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return np.hstack([np.asarray(input_ids, dtype=np.int32)]+ 
            [np.asarray(input_masks, dtype=np.int32)]+
            [np.asarray(input_segments, dtype=np.int32)])

In [79]:
NUM_FOLDS=4
kf = MultilabelStratifiedKFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)


y_train = df_train[df_train.columns[11:]].values
X_train = df_train[df_train.columns[:11]]

X_train2 = hstack_input_arays(X_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

df_spearman_stkfold = pd.DataFrame(df_train.columns[11:],columns=["Label"])

for ind, (tr, val) in enumerate(kf.split(X_train2, y_train)):
    #X_tr = X_train[tr]
    #y_tr = y_train[tr]
    X_vl = X_train.loc[val]
    y_vl = y_train[val]

    x_input = compute_input_arays(X_vl,input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

    df_valid_pred = bagged_model(x_input)
    
    coefs = []

    for i in range(30):
      coef,p = spearmanr(df_valid_pred[:,i], y_vl[:,i])
      coefs.append(coef)

    colname = 'Score'+str(ind)
    df_spearman_stkfold[colname] = coefs
           

6079it [00:34, 176.17it/s]
1522it [00:08, 176.59it/s]


1522/1522 [==============================] - 37s 24ms/sample


1509it [00:08, 181.28it/s]


1509/1509 [==============================] - 37s 24ms/sample


1535it [00:08, 178.36it/s]


1535/1535 [==============================] - 37s 24ms/sample


1513it [00:08, 180.27it/s]


1513/1513 [==============================] - 37s 24ms/sample


In [80]:
df_spearman_stkfold

,Label,Score0,Score1,Score2,Score3
0,question_asker_intent_understanding,0.533462,0.524203,0.542171,0.509370
1,question_body_critical,0.747058,0.752342,0.726799,0.722786
2,question_conversational,0.478382,0.463704,0.463265,0.444813
3,question_expect_short_answer,0.596599,0.650699,0.600397,0.586654
4,question_fact_seeking,0.672332,0.681383,0.678991,0.647743
5,question_has_commonly_accepted_answer,0.652439,0.632285,0.604023,0.646079
6,question_interestingness_others,0.471137,0.492640,0.495568,0.474275
7,question_interestingness_self,0.626630,0.645253,0.637935,0.627667
8,question_multi_intent,0.720586,0.725564,0.713097,0.732977
9,question_not_really_a_question,0.147880,0.128152,0.121515,0.121788


In [81]:
df_spearman_stkfold.mean().mean()

0.5495676537054117

Using Stratified KFold improved our Spearman scores from 

## Run the model on the test set

In [82]:
final_predictions = bagged_model(test_inputs)

476/476 [==============================] - 12s 24ms/sample


In [0]:
df_sub.iloc[:, 1:] = final_predictions
df_sub.to_csv('submission.csv', index=False)

In [84]:
df_sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.922638,0.567927,0.064251,0.608264,0.711082,0.850872,0.713220,0.661823,0.860829,0.003948,0.644606,0.834519,0.008475,0.275376,0.010538,0.011853,0.084039,0.038577,0.655305,0.002930,0.832237,0.933127,0.646431,0.979862,0.946747,0.730714,0.040916,0.015051,0.853433,0.913076
1,46,0.852433,0.395800,0.004266,0.820851,0.694166,0.943101,0.580638,0.485877,0.019464,0.006713,0.423158,0.118904,0.001748,0.001615,0.001691,0.002816,0.914795,0.052117,0.054587,0.000704,0.499841,0.950354,0.667901,0.982334,0.981792,0.893747,0.966207,0.049795,0.029797,0.919274
2,70,0.842327,0.506961,0.004563,0.883937,0.932034,0.988546,0.683724,0.545802,0.043648,0.004434,0.114470,0.761507,0.004864,0.022064,0.003058,0.002728,0.054134,0.014959,0.755193,0.001444,0.723875,0.945495,0.616236,0.971957,0.981394,0.751724,0.079267,0.034405,0.811195,0.902751
3,132,0.882259,0.339376,0.004279,0.635084,0.730497,0.905576,0.589358,0.443165,0.073747,0.011523,0.526853,0.013763,0.002484,0.006580,0.002225,0.001478,0.930140,0.073568,0.602392,0.000985,0.725173,0.925942,0.674307,0.961560,0.980880,0.875148,0.924776,0.073127,0.500388,0.914971
4,200,0.872123,0.286056,0.008000,0.878443,0.790840,0.967963,0.666880,0.658153,0.022098,0.007939,0.266337,0.121490,0.003425,0.006583,0.005576,0.024539,0.229945,0.082714,0.453629,0.001472,0.465335,0.943488,0.666579,0.979266,0.958229,0.826378,0.330356,0.141307,0.694931,0.894270
